# Library Management system

In [39]:
"""Importing mysql, pandas, auth"""
from mysql import connector
import pandas as pd
import os
import auth

In [36]:
con = connector.connect(user=auth.user,password=auth.password,host=auth.host,database=auth.database)
my_cursor = con.cursor()

## Issue and Return book


In [20]:
def issue_book():
    """issue book function  create the record to the table book_issue
    and updates books table."""
    query_reader = pd.read_sql_query('select * from reader', con)
    print('Table : reader')
    display(query_reader)
    print('Table : books')
    display(pd.read_sql_query('select * from books', con))

    user_id = int(input('Enter user ID : '))

    if user_id in query_reader['r_id'].to_list():
        book_id = input('Enter the book id : ')
        if int(book_id) not in \
                pd.read_sql_query('select b_id from book_issue', con)['b_id'].to_list():
            my_cursor.execute(f'INSERT INTO book_issue VALUES({user_id},{book_id});')
            my_cursor.execute(f"update books set b_availability = 'Issued' where b_id = {book_id};")
            con.commit()
            print('Book Successfully updated.\n')
        else:
            print('Book is not available for issue.\n')
    else:
        print('User not found!\n')

In [21]:
def return_book():
    """return book function returns the delete the record from the table book_issue
    and updates books table."""
    print('Table : book_issue')
    display(pd.read_sql_query('select * from book_issue', con))

    user_id = int(input('Enter user ID : '))
    if user_id not in pd.read_sql_query('select * from book_issue', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    else:
        book_id = int(input('Enter the book id : '))
        if book_id not in pd.read_sql_query('select * from book_issue', con)['b_id'].to_list():
            print('Book ID is wrong.\n')

        else:
            my_cursor.execute(f'DELETE from book_issue where b_id ={book_id} and r_id={user_id}')
            my_cursor.execute(f'update books set b_availability = \'Available\' where b_id ={book_id}')
            con.commit()
            print('Book Successfully returned.\n')

## Viewing records of the tables

In [40]:
def table_books():
    """Views table books."""
    print('Table : books')
    display(pd.read_sql_query('select * from books', con))
    os.system('pause')

In [41]:
def table_reader():
    """Views table reader."""
    print('Table : reader')
    display(pd.read_sql_query('select * from reader', con))
    os.system('pause')

In [42]:
def table_book_issue():
    """Views table book_issue."""
    print('Table : book_issue')
    display(pd.read_sql_query('select * from book_issue', con))
    os.system('pause')

## Upload records in the table

In [25]:
def add_new_reader():
    """add new book reader adds new reader record to the tables reader."""
    display(pd.read_sql_query('select * from reader', con))
    reader_id, reader_name = int(
        pd.read_sql_query('''
        select r_id from reader order by r_id desc limit 1
        ''', con).values[0][0]) + 1, input(
        'Enter name of the reader : ')
    try:
        my_cursor.execute(f'insert into reader values({reader_id},\'{reader_name}\');')
        con.commit()
        print(f'Successfully added the book {reader_name} to table reader.\n')
    except connector.Error as error:
        print('Update failed!\n')
        print(f'Error : {error}')

In [26]:
def add_new_book():
    """add new book function adds new book record to the tables book."""
    display(pd.read_sql_query('select * from books', con))
    b_id = int(pd.read_sql_query('''
    select b_id from books order by b_id desc limit 1
    ''', con).values[0][0]) + 1
    b_name = input('Enter the name of the book : ')
    b_author = input('Enter the name of the Author : ')
    b_genre = input('Enter the name of the Genre : ')
    try:
        my_cursor.execute(f"insert into books values({b_id}, '{b_name}', '{b_author}', '{b_genre}', 'Available')")
        con.commit()
        print(f'Successfully added {b_name} user to table reader.\n')
    except connector.Error as error:
        print(f'Error : {error}')
        print('Update failed!\n')

## Modify records in the tables

In [27]:
def modify_reader():
    """modify reader function modifies the reader record from the tables reader."""
    display(pd.read_sql_query('select * from reader', con))
    modify_readers = int(input('enter the reader id'))
    if modify_readers not in pd.read_sql_query('select * from reader', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    else:
        r_name = input('Enter new name of the reader')
        my_cursor.execute(f"update reader set r_name = '{r_name}'where r_id = {modify_readers};")
        con.commit()
        print('Reader Successfully modified.\n')

In [28]:
def modify_book():
    """modify book function modifies the book record from the tables book."""
    display(pd.read_sql_query('select * from books', con))
    modify_books = int(input('enter the book id'))
    if modify_books not in pd.read_sql_query('select * from books', con)['b_id'].to_list():
        print('Book ID is wrong.\n')
    else:
        b_name, b_author, b_genre = input(
            'Enter new name of the book'), input(
            'Enter new Author name of the book'), input(
            'Enter new Genre of the book')
        my_cursor.execute(
            f'''
            update books 
            set b_name = '{b_name}', 
            b_author = '{b_author}', 
            b_genre = '{b_genre}' 
            where b_id = {modify_books};
            ''')
        con.commit()
        print('Book Successfully modified.\n')

## Remove records from tables

In [29]:
def remove_book():
    """remove book function removes the book record from the tables book."""
    display(pd.read_sql_query('select * from books', con))

    user_id = int(input('Enter user ID : '))

    if user_id not in pd.read_sql_query('select * from book_issue', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    else:
        remove_books: int = int(input('enter the book id'))
        if remove_books not in pd.read_sql_query('select * from books', con)['b_id'].to_list():
            print('Book ID is wrong.\n')
        else:
            query = pd.read_sql_query(f'select * from books where b_id = {remove_books}', con)
            if (query['b_availability'] == 'Issued')[0]:
                print('Issued book cannot be removed.\n')
            else:
                my_cursor.execute(f'delete from books where b_id = {remove_books}')
                con.commit()
                print('Book Successfully Deleted.\n')

In [30]:
def remove_reader():
    """remove reader function removes the reader record from the tables reader."""
    display(pd.read_sql_query('select * from reader', con))
    remove_user = int(input('enter the user id'))
    if remove_user not in pd.read_sql_query('select * from reader', con)['r_id'].to_list():
        print('User ID is wrong.\n')
    elif remove_user in pd.read_sql_query('select r_id from book_issue', con)['r_id'].to_list():
        print('User cannot be removed before returning the book.\n')
    else:
        my_cursor.execute(f'delete from reader where r_id = {remove_user}')
        con.commit()
        print('User Successfully Deleted.\n')

## Main program

In [31]:
def select_table(fn1, fn2, fn3=None):
    """Select function use to get the tables names."""
    print([
              'Please select a table :\n1. Books\n2. Reader' \
                  if fn3 is None else 'Please select a table :\n1. Books\n2. Reader\n3. Book issue'][0])
    user_input_remove = input()
    if user_input_remove == '1':
        fn1()
    elif user_input_remove == '2':
        fn2()
    elif user_input_remove == '3':
        fn3()
    else:
        print('Wrong option!.\n')

In [46]:
def main():
    USER_INPUT = None
    while True:
        print('Welcome to the Library management system')
        print(
            '''\nPlease choose an option from below menu:\n1. Issue a book\n2. Return a book\n3. View records\n4. Upload records\n5. Remove records\n6. Modify records\n7. Exit\n''')

        try:
            USER_INPUT = int(input())
        except connector.Error as e:
            print('Entered input is not a number please try again.\n')
            print(f'Error : {e}')

        # Issue a book
        if USER_INPUT == 1:
            issue_book()
        # Return a book
        elif USER_INPUT == 2:
            return_book()
        # View records
        elif USER_INPUT == 3:
            select_table(table_books, table_reader, table_book_issue)
        #  Update records
        elif USER_INPUT == 4:
            select_table(add_new_book, add_new_reader)
        # Remove records
        elif USER_INPUT == 5:
            select_table(remove_book, remove_reader)
        # modify records
        elif USER_INPUT == 6:
            select_table(modify_book, modify_reader)
        # exit
        elif USER_INPUT == 7:
            con.close()
            break

In [ ]:
# main()

## References

[MySQL Connectors](https://www.mysql.com/products/connector/)<br>
[MySQL Error Exception](https://dev.mysql.com/doc/connector-python/en/connector-python-api-errors-error.html)<br>
[Pandas .read_sql_query](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)